In [3]:
import psycopg2
import pandas as pd
from datetime import datetime
import random

In [127]:
df = pd.read_csv("./dentist_rand_sched_utf.csv")

In [ ]:
df.head()

,dentist_name,clinic_name,address,city_town,province,region,zip_code,contact_number,Monday_start,Monday_end,...,Wednesday_end,Thursday_start,Thursday_end,Friday_start,Friday_end,Saturday_start,Saturday_end,Sunday_start,Sunday_end,id
0,AIDA MAITIM JALUAG,A.M. JALUAG DENTAL CLINIC,"G/F UNIT B-3 LEGASPI TOWER 300, 2600 ROXAS BLV...",MANILA,METRO MANILA,NCR,1004,5313-8506 / (0905) 433-1147,8,17,...,17,8,17,8,17,NaN,NaN,NaN,NaN,4838
1,ANGE LUMAWAG,L & S DENTAL CARE CLINIC,"7/F VIP BLDG., 1140 ROXAS BOULEVARD, ERMITA",MANILA,METRO MANILA,NCR,1000,7617-7079 / (0917) 546-6292,9,18,...,18,9,18,9,18,8.0,17.0,8.0,17.0,4839
2,MARJORIE JOY LIM,DENTAL WELLNESS STUDIO,"UNIT GD BURGUNDY WEST BAY TOWER, 820 PABLO OCA...",MANILA,METRO MANILA,NCR,1004,5313-5746 / (0998) 573-3788,10,19,...,19,10,19,10,19,7.0,11.0,7.0,11.0,4840
3,CONCEPCION PONCE PEÑAFLORIDA,PONCE PEÑAFLORIDA DENTAL CLINIC,"RM. 1, 2/F ARV BLDG., 1728 RIZAL AVENUE, STA. ...",MANILA,METRO MANILA,NCR,1014,8741-7260 / 7577-1833 / (0917) 516-1066,8,17,...,17,8,17,8,17,9.0,13.0,9.0,13.0,4841
4,MARISSA LOMOTAN,MARISSA ORIO-LOMOTAN DENTAL CENTRAL,4747 OLD STA. MESA,MANILA,METRO MANILA,NCR,1016,7219-3617 / (0917) 147-5214,10,19,...,19,10,19,10,19,NaN,NaN,NaN,NaN,4842


In [ ]:
# get user's location and search all dentist in that area
user_city = "caloocan"
# display clinics in that area
dentists = df.loc[df['city_town'].str.contains(user_city.lower(),case=False)]
dentists = dentists['clinic_name'].to_string(index=False)
dentist_list = dentists.strip().replace('\\n',"\n")
print(dentist_list)

         PUJALTE DENTAL CLINIC
         EZETHEA DENTAL CLINIC
           C. ASTOR DENTAL HUB
     TOOTH CHECK DENTAL CLINIC
      MJCR ORTHO DENTAL CLINIC
       GC TEJADA DENTAL CLINIC
           RJKDC DENTAL CLINIC
DRK NAVARRO DENTAL CARE CLINIC


In [ ]:
# choose what clinic and return id of selected clinic
clinic_name = "PUJALTE DENTAL CLINIC"
clinic_id = df.loc[df['clinic_name'].str.contains(clinic_name.lower(),case=False)]['id'].to_string(index=False)
print(clinic_id)

4849


In [ ]:
# select date of appointment
date_today = datetime.now()
user_selected = "12/24/2023" # accepts mm/dd/yyyy format only for now, use date picker if available
date_format = '%m/%d/%Y'
user_date = datetime.strptime(user_selected, date_format)

# check if date is done or not
if user_date < date_today:
    print("cant select that date")
else:
    # check if selected clinic can accomodate that day
    sch_day = user_date.strftime('%A')
    time_start = df.loc[df['id'] == int(clinic_id)][f'{sch_day}_start'].values
    print(time_start)
    can_accomodate = time_start[0]>0

[9.]


In [ ]:
# select time of appointment (in military time)
user_time = 9
user_id = random.randint(0,20) # wala ako access haha
if can_accomodate:
    # making sure selected time is within operating hours
    if time_start[0] > user_time:
        print("selected time is outside of clinic hours")
    else:
        selected_start_time = user_time
        stop_time = selected_start_time + 1

# convert everything to a sql statement:
sql_query = f'''INSERT INTO app_dentist_schedule (clinic_id, user_id, appointment_date, "start", stop) 
            VALUES ({clinic_id}, {user_id}, '{user_date.date().strftime('%Y-%m-%d')}', {selected_start_time}, {stop_time})'''
print(sql_query)


INSERT INTO app_dentist_schedule (clinic_id, user_id, appointment_date, "start", stop) 
            VALUES (4849, 3, '2023-12-24', 9, 10)


In [ ]:
# need to add reference number, service, 

CONVERT THE SELECTION TO PYTHON FUNCTIONS

In [12]:
from dotenv import dotenv_values

# loading the credentials
db_creds = dotenv_values("../.env")
host = db_creds['URL']
user = db_creds['USER']
password = db_creds['PASSWORD']
dbname = db_creds['DB']

In [17]:
import datetime
import psycopg2

def generate_ctrl_num(date):
    """Accepts date and returns a control number. Add 1 to the int part of the control number if existing in DB."""

    # sample = QCYYYYMMDD001, QC20231203001
    date_part = date.strftime("%Y%m%d")

    # check db if how many control number on a certain date
    sql_query = f"SELECT COUNT(*) FROM app_dentist_schedule WHERE reference_number ILIKE '%QC{date_part}%';"
    conn = psycopg2.connect(dbname=dbname, user=user, host=host, password=password)
    cur = conn.cursor()
    cur.execute(sql_query)
    ans = cur.fetchall()

    # add 1 to count 
    num = int(ans[0][0]) + 1
    num_part = str(num).rjust(3, "0")
    return f'QC{date_part}{num_part}'

In [8]:
print(datetime.datetime(2023,12,24).date())

2023-12-24


In [18]:
samp_date = datetime.datetime(2023,12,24).date()
generate_ctrl_num(samp_date)

[(0,)]
QC20231224001


In [ ]:
insert into app_dentist_schedule (clinic_id, user_id, appointment_date, "start", stop, procedure_type, reference_number)
values (4849, 1234, '2023-12-23', 9, 10, 'Fillings', 'QC20231223001');

In [3]:
def paginator(array):
    """This function accepts array and will return a string. Displays only 5 per page. Returns string value of the selected option from array."""

    if len(array) == 5:
        choices = []
        for i in range(len(array)):
            print(f"{i} | {array[i]}")
            choices.append(array[i].lower())

        x = ''
        cont = True
        while cont:
            x = input("Type the number or value to continue")

            if x.lower() == 'q':
                cont = False
                print('Selecting aborted')
                break
            elif type(x) == str and x.lower() in choices:
                print(f'You selected {x}')
                cont = False
                return x.upper()
            else:
                try: 
                    x = int(x)
                    if choices[x] in choices:
                        print(f'You selected {choices[x].upper()}')
                        cont = False
                        return choices[x].upper()
                except:
                    pass

    else:
        n = (len(array) // 5)
        if (len(array) % 5) == 0:
            n_page = n
        else:
            n_page = n + 1

        choices = []
        cur_pos = 0

        for i in range(n_page + 1):
            if i == n_page:
                s = slice(cur_pos, -1)
                sub_choice = array[s]
                choices.append(sub_choice)
            else:
                sub_choice = []
                s = slice(cur_pos, cur_pos+5)
                sub_choice = array[s]
                choices.append(sub_choice)
                cur_pos += 5

        curr_page = 0
        print(f'{curr_page+1}/{n_page} page - {len(array)} options \n')

        for i in range(len(choices[curr_page])):
            print(f"{i} | {choices[curr_page][i]}")

        print("\n")
    
        x = ''
        cont = True
        while cont:
            x = input("Type the number or value to continue. Type 'next page' or 'previous page' to change options.\n")

            if x.lower() == 'q':
                cont = False
                print('Selecting aborted')
                break
            elif x.lower() == 'next page':
                if (len(array) % 5) == 0:
                    if curr_page >= (len(array) // 5) -1:
                        print("This is the last page")
                    elif curr_page < n_page:
                        curr_page += 1
                        print(f'{curr_page+1}/{n_page} page - {len(array)} options \n')

                        for i in range(len(choices[curr_page])):
                            print(f"{i} | {choices[curr_page][i]}")
                        print("\n")
                else:
                    if curr_page >= (len(array) // 5):
                        print("This is the last page")
                    elif curr_page < n_page:
                        curr_page += 1
                        print(f'{curr_page+1}/{n_page} page - {len(array)} options \n')

                        for i in range(len(choices[curr_page])):
                            print(f"{i} | {choices[curr_page][i]}")
                        print("\n")
            elif x.lower() == 'previous page':
                if curr_page > 0:
                    curr_page -= 1
                    print(f'{curr_page+1}/{n_page} page - {len(array)} options \n')

                    for i in range(len(choices[curr_page])):
                        print(f"{i} | {choices[curr_page][i]}")
                    print("\n")
                else:
                    print("This is the first page")
            elif type(x) == str  and x.upper() in choices[curr_page]:
                print(f'You selected {x}')
                cont = False
                return x.upper()
            else:
                try: 
                    x = int(x)
                    if choices[curr_page][x] in choices[curr_page]:
                        print(f'You selected {choices[curr_page][x].upper()}')
                        cont = False
                        return choices[curr_page][x].upper()
                except:
                    pass

In [198]:
test_array = ["A","B","C","D","E","F", "G", "H", "I", "J", "K"]
prodedures = ["Dental Cleanings", "Fillings", "Root Canal Therapy", "Tooth Extraction", "Crowns", "Bridges", "Dental Implants", "Dentures", "Teeth Whitening", "Dental Checkup"]
test_array_2 = ["A","B","C","D","E"]

In [13]:
import pandas as pd

def city_selector(location):
    """Takes the user's city as parameters. Returns array of dentists based on user's selected city."""
    
    # loading csv
    df = pd.read_csv("./dentist_rand_sched_utf.csv")

    while True:
        x = input(f"Current location is {location}. Type 'Y' to proceed or type another city.")
        
        if x.lower == 'q':
            break
        elif x.lower() == 'y':
            dentists = df.loc[df['city_town'].str.contains(location.lower(),case=False)] # not exact match for now
            return dentists['clinic_name'].values.tolist()
        else:
            try:
                dentists = df.loc[df['city_town'].str.contains(x.lower(),case=False)] 
                location = x
            except:
                print("Cannot find dentist in your location. Print another location")

In [14]:
city_selector("Manila")

['PUJALTE DENTAL CLINIC',
 'EZETHEA DENTAL CLINIC',
 'C. ASTOR DENTAL HUB',
 'TOOTH CHECK DENTAL CLINIC',
 'MJCR ORTHO DENTAL CLINIC',
 'GC TEJADA DENTAL CLINIC',
 'RJKDC DENTAL CLINIC',
 'DRK NAVARRO DENTAL CARE CLINIC']

In [8]:
def dentist_selector(dentist_list):
    print("Please select a clinic \n")

    dentist = paginator(dentist_list)
    return dentist  

In [6]:
def procedure_selector():
    procedure_list = ["Dental Cleanings", "Fillings", "Root Canal Therapy", "Tooth Extraction", "Crowns", "Bridges", "Dental Implants", "Dentures", "Teeth Whitening", "Dental Checkup"]

    procedure = paginator(procedure_list)
    return procedure

In [7]:
def get_clinic_id(clinic_name, city):
    """Accepts clinic name and city as parameters. Returns id of the said clinic in the selected city."""
    clinic_name = clinic_name.strip()
    # loading csv
    df = pd.read_csv("./dentist_rand_sched_utf.csv")
    
    try:
        clinic_id = df.loc[(df['clinic_name'].str.contains(clinic_name.lower(),case=False)) & df['city_town'].str.contains(city.lower(),case=False)]['id'].to_string(index=False)
        return int(clinic_id)
    except:
        return "Cannot find selected clinic"

In [5]:
def date_selector(clinic_id):
    """Must input clinic id as parameter. Displays clinic's operation hours. Returns date in %Y-%m-%d, start_time and end_time."""

    df = pd.read_csv("./dentist_rand_sched_utf.csv")
    
    # select date of appointment
    date_today = datetime.now()
    # user_selected = "12/24/2023"
    # # accepts mm/dd/yyyy format only for now, use date picker if available
    date_format = '%m/%d/%Y'

    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    print("Operating hours:")
    for day in days:
        time_start = df.loc[df['id'] == int(clinic_id)][f'{day}_start'].values
        if time_start >= 0:
            time_end = df.loc[df['id'] == int(clinic_id)][f'{day}_end'].values
            print(f"{day} | {int(time_start[0])}:00 - {int(time_end[0])}:00")

    while True:
        user_selected = input("Please type desired date in mm/dd/yyyy format.")
        # check if date is done or not
        try:
            user_date = datetime.strptime(user_selected, date_format)
            if user_date < date_today:
                print("cant select that date") # change error message para maganda
            else:
                # check if selected clinic can accomodate that day
                sch_day = user_date.strftime('%A')
                # time_start = df.loc[df['id'] == int(clinic_id)][f'{sch_day}_start'].values

                # user_time = str(input('Select desired time'))
                # if user_time >= time_start:
                #     selected_start = user_time
                # else:
                #     print("Please select a valid time")
                return user_date.date()
        except:
            if user_selected.lower() == 'q':
                break
            else:
                print("Not a date bro") # Change message here

In [10]:
x = date_selector(4849)

Operating hours:
Monday | 8:00 - 17:00
Tuesday | 8:00 - 17:00
Wednesday | 8:00 - 17:00
Thursday | 8:00 - 17:00
Friday | 8:00 - 17:00
Saturday | 9:00 - 18:00
Sunday | 9:00 - 18:00


In [8]:
def time_selector(date, clinic_id):
    df = pd.read_csv("./dentist_rand_sched_utf.csv")
    day = date.strftime('%A')
    time_start = df.loc[df['id'] == int(clinic_id)][f'{day}_start'].values

    while True:
        user_time = input("Type desired time. Use 24 hr format and omit the :00")
        try:
            user_time = int(user_time)
            if int(user_time) >= time_start:
                return user_time
            elif int(user_time) < time_start:
                print(f'Operating hours on {day}s starts at {int(time_start[0])}:00')
        except:
            print("Please type in integer only.")

In [11]:
time_selector(x, 4849)

Operating hours on Saturdays starts at 9:00
Operating hours on Saturdays starts at 9:00
Please type in integer only.


9

In [11]:
def set_appointment():
    """Sets appointment using series of questions. Returns sql INSERT statement."""
    
    city = "Manila"
    procedure = procedure_selector()
    print("\n")
    clinic_list = city_selector(city)
    print("\n")
    clinic = dentist_selector(clinic_list)
    print("\n")
    clinic_id = get_clinic_id(clinic, city)
    print("\n")
    apt_date = date_selector(clinic_id)
    print("\n")
    time_start = time_selector(apt_date, clinic_id)

    print(f"""APPOINTMENT SUMMARY:
{procedure} at {clinic} on {apt_date.strftime('%m/%d/%Y')}, {time_start}:00 - {time_start + 1}:00""")